In [3]:
import math
import numpy as np
import wandb
from tqdm import tqdm
from torchvision import transforms
from utils import get_data, accuracy, save_Yte, augment_dataset, array_to_tensor
from kernels import *
from models import MultiKRR
from feature import hog
from sklearn.model_selection import train_test_split 


In [4]:
Xtr, Xte, Ytr = get_data()

## Hyperparameter tuning

In [5]:
Xtr_, Xval_, Ytr_, Yval_ = train_test_split(Xtr, Ytr, test_size=0.1, random_state=123)
Xtr_tensor_ = array_to_tensor(Xtr_)
Xval_tensor_ = array_to_tensor(Xval_)

In [6]:
# test
hog_fun = lambda img: hog(img)
Xtr_hog_ = np.array([hog_fun(img) for img in tqdm(Xtr_tensor_, desc="Computing HOG")])
Xval_hog_ = np.array([hog_fun(img) for img in Xval_tensor_])
krr = MultiKRR(kernel=GaussianKernel(sigma=1).kernel, lambd=1e-4)
krr.fit(Xtr_hog_, Ytr_)
print("accuracy:", accuracy(Yval_, krr.predict(Xval_hog_)))

Computing HOG for train:   1%|          | 39/4500 [00:00<00:23, 192.10it/s]

computing HOG


Computing HOG for train: 100%|██████████| 500/500 [00:03<00:00, 163.80it/s]


regression


Fitting: 100%|██████████| 10/10 [00:53<00:00,  5.32s/it]

accuracy: 0.6


In [6]:
def get_sweep_id(parameters):
    """https://docs.wandb.ai/guides/sweeps/configuration"""
    sweep_config = {
        'method': "bayes",
        'metric': {
          'name': 'accuracy',
          'goal': 'maximize'
        },
        'parameters': parameters
    }
    sweep_id = wandb.sweep(sweep_config, project='kernel-challenge-final')

    return sweep_id

In [7]:
import math

def one_run():
    wandb.init(project='kernel-challenge-final', entity='theodumont', tags=["HOG+KRR"])
    # dataset ============================================================================
    hog_fun = lambda img: hog(img,
        cells_per_block=wandb.config.cells_per_block,
        normalization=wandb.config.normalization,
    )
    Xtr_hog_ = np.array([hog_fun(img) for img in tqdm(Xtr_tensor_, desc="Computing HOG")])
    Xval_hog_ = np.array([hog_fun(img) for img in Xval_tensor_])

    # kernel =============================================================================
    kernel_name = wandb.config.kernel_name
    if kernel_name == "GaussianKernel":
        kernel = GaussianKernel(sigma=wandb.config.sigma).kernel
    elif kernel_name == "LogKernel":
        pass
    elif kernel_name == "MinKernel":
        pass
    elif kernel_name == "Chi2Kernel":
        pass
    
    # model ==============================================================================
    krr = MultiKRR(
        kernel=kernel,
        lambd=wandb.config.lambd,
    )
    krr.fit(Xtr_hog_, Ytr_)
    wandb.run.summary["accuracy"] = accuracy(Yval_, krr.predict(Xval_hog_))

In [8]:
parameters = {
    'lambd': {
        'distribution': 'log_uniform',
        'min': math.log(1e-5),
        'max': math.log(1e-3),
    },
    'cells_per_block': {
        'values': [2, 3],
    },
    'normalization': {
        'values': ['L1', 'L2-Hys'],
    },
    # kernel -----------------------------------------------------------------------------
    'kernel_name': {
        'value': "GaussianKernel",
        # 'values': ["GaussianKernel", "LogKernel", "MinKernel", "Chi2Kernel"],
    },
    'sigma': {
        'distribution': 'log_uniform',
        'min': math.log(1e-1),
        'max': math.log(1e1),
    },
}
sweep_id = get_sweep_id(parameters)
%env WANDB_SILENT=True

wandb.agent(sweep_id, function=one_run)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 2rx2v24s
Sweep URL: https://wandb.ai/theodumont/kernel-challenge-final/sweeps/2rx2v24s
env: WANDB_SILENT=True


wandb: Agent Starting Run: szqs8hkz with config:
wandb: 	cells_per_block: 2
wandb: 	kernel_name: GaussianKernel
wandb: 	lambd: 3.51732841536161e-05
wandb: 	normalization: L2-Hys
wandb: 	sigma: 1.1150743436996975
Fitting:   0%|          | 0/10 [00:00<?, ?it/s]